In [1]:
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig
)
import torch

In [2]:
use_4bit = True 
bnb_4bit_compute_dtype = "float16" 
bnb_4bit_quant_type = "nf4" 
use_nested_quant = False 

In [3]:
compute_dtype = getattr(torch, bnb_4bit_compute_dtype)
bnb_config = BitsAndBytesConfig(
    load_in_4bit=use_4bit,
    bnb_4bit_quant_type=bnb_4bit_quant_type,
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_use_double_quant=use_nested_quant,
)

In [4]:
model = AutoModelForCausalLM.from_pretrained(
    "WinF/llama-2-7b-cloudformation",
    device_map={"": 0}, quantization_config=bnb_config,
)
tokenizer = AutoTokenizer.from_pretrained("WinF/llama-2-7b-cloudformation", trust_remote_code=True)

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [20]:
next(model.parameters()).device

device(type='cuda', index=0)

In [5]:
model.eval()
model.config.use_cache = (True)
tokenizer.pad_token = tokenizer.eos_token

In [6]:
device = (
        torch.device("cuda:0") if torch.cuda.is_available() else torch.device("cpu")
    )

In [7]:
prompt = """
<s>[INST]너는 지금부터 아래 내용에 따라 AWS CloudFormation JSON 템플릿을 만드는 역할을 할거야. 너한테 AWS CloudFormation의 템플릿 파일을 생성하기 위해 지시를 내리는 문장과 지시 문장과 관련된 AWS의 CloudFormation의 문서를 제공할거야. - AWS CloudFormation 템플릿의 예시: AWS CloudFormation 템플릿 형태에 대해 설명해줄게.    템플릿 구조    JSON    {    "AWSTemplateFormatVersion" : "version date",    "Description" : "JSON string",    "Resources" : {      set of resources    }  }  템플릿 섹션  AWSTemplateFormatVersion  AWSTemplateFormatVersion 섹션은 템플릿의 기능을 식별합니다. 최신 템플릿 포맷 버전은 2010-09-09이며 현재 유일한 유효 값입니다.  Description  템플릿의 Description 섹션(선택 사항)에 템플릿에 대한 설명을 지정합니다. 설명 선언 값은 0 ~ 1023바이트 길이의 리터럴 문자열이어야 합니다. 파라미터나 함수를 사용하여 설명을 지정할 수 없습니다. 다음 코드 조각은 설명 선언을 보여주는 예입니다.  Resources  필수 Resources 섹션은 Amazon EC2 인스턴스 또는 Amazon S3 버킷 등 스택에 포함시킬 AWS 리소스를 선언합니다.  Resources 섹션은 키 이름 Resources로 이루어집니다. 다음 가상 템플릿에는 Resources 섹션이 요약되어 있습니다.  JSON  "Resources" : {      "Logical ID" : {          "Type" : "Resource type",          "Properties" : {              Set of properties          }      }}      - 지시 문장의 예시EC2 인스턴스용 CloudFormation 템플릿을 생성하되, AMI ID를 ami-1234567890abcdef0로 설정해주고 인스턴스 타입을 t2.micro로 설정해줘- AWS CloudFormation의 문서 예시    Specifies an EC2 instance.    If an Elastic IP address is attached to your instance, AWS CloudFormation reattaches the Elastic IP address after it updates the instance. For more information about updating stacks, see AWS CloudFormation Stacks Updates.    To declare this entity in your AWS CloudFormation template, use the following syntax:    JSON    {      "Type" : "AWS::EC2::Instance",      "Properties" : {          "AvailabilityZone" : String,          "ImageId" : String,          "InstanceType" : String        }    }    Properties    AvailabilityZone    The Availability Zone of the instance.    If not specified, an Availability Zone will be automatically chosen for you based on the load balancing criteria for the Region.    This parameter is not supported by DescribeImageAttribute.    Required: No    Type: String    Update requires: Replacement    ImageId    The ID of the AMI. An AMI ID is required to launch an instance and must be specified here or in a launch template.    Required: Conditional    Type: String    Update requires: Replacement    InstanceType    The instance type. For more information, see Instance types in the Amazon EC2 User Guide.    When you change your EBS-backed instance type, instance restart or replacement behavior depends on the instance type compatibility between the old and new types. An instance with an instance store volume as the root volume is always replaced. For more information, see Change the instance type in the Amazon EC2 User Guide.    Required: No    Type: String    Allowed values: r6i.8xlarge | r6i.12xlarge | r6i.16xlarge | r6i.24xlarge | r6i.32xlarge | r6i.metal | t1.micro | t2.nano | t2.micro | t2.small | t2.medium | t2.large | t2.xlarge | t2.2xlarge | t3.nano    Update requires: Some interruptions - 너가 생성할 JSON 템플릿의 예시: JSON 템플릿은 아래와 같이 JSON 형식으로 만들어질거야.{  "AWSTemplateFormatVersion": "2010-09-09",  "Description": "CloudFormation Template for EC2 Instance with specific AMI and instance type",  "Resources": {  "Ec2Instance": {    "Type": "AWS::EC2::Instance",    "Properties": {        "ImageId": "ami-1234567890abcdef0",        "InstanceType": "t2.micro"      }    }  }}  세개의 백틱 안에 들어있는 내용은 지시 문장과 AWS CloudFormation의 문서야. 이 문서의 내용을 토대로 JSON 템플릿의 예시로 보여준거처럼 JSON 템플릿을 만들어줘.```API 게이트웨이 키를 생성하되, 이름을 'TestApiKey', 설명을 'CloudFormation API Key V1'로 설정해줘. 또한 활성화 상태를 true로 설정하고, 'Test' 단계의 'TestAPIDeployment' 배포와 연관시켜줘- AWS CloudFormation의 문서# AWS::ApiGateway::ApiKeyThe `AWS::ApiGateway::ApiKey` resource creates a unique key that you can distribute to clients who are executing API Gateway `Method` resources that require an API key\. To specify which API key clients must use, map the API key with the `RestApi` and `Stage` resources that include the methods that require a key\. ## SyntaxTo declare this entity in your AWS CloudFormation template, use the following syntax:### JSON```{  "Type" : "AWS::ApiGateway::ApiKey",  "Properties" : {      "CustomerId" : String,      "Description" : String,      "Enabled" : Boolean,      "GenerateDistinctId" : Boolean,      "Name" : String,      "StageKeys" : [ StageKey, ... ],      "Tags" :  [Tag, ... ],      "Value" : String    }}```## Properties`CustomerId`  An AWS Marketplace customer identifier, when integrating with the AWS SaaS Marketplace\.  *Required*: No  *Type*: String  *Update requires*: No interruption`Description`  The description of the ApiKey\.  *Required*: No  *Type*: String  *Update requires*: No interruption`Enabled`  Specifies whether the ApiKey can be used by callers\.  *Required*: No  *Type*: Boolean  *Update requires*: No interruption`GenerateDistinctId`  Specifies whether \(`true`\) or not \(`false`\) the key identifier is distinct from the created API key value\. This parameter is deprecated and should not be used\.  *Required*: No  *Type*: Boolean  *Update requires*: Replacement`Name`  A name for the API key\. If you don't specify a name, AWS CloudFormation generates a unique physical ID and uses that ID for the API key name\. For more information, see Name Type\.  If you specify a name, you cannot perform updates that require replacement of this resource\. You can perform updates that require no or some interruption\. If you must replace the resource, specify a new name\. *Required*: No  *Type*: String  *Update requires*: Replacement`StageKeys`  DEPRECATED FOR USAGE PLANS \- Specifies stages associated with the API key\.  *Required*: No  *Type*: List of StageKey  *Update requires*: No interruption`Tags`  The key\-value map of strings\. The valid character set is \[a\-zA\-Z\+\-=\.\_:/\]\. The tag key can be up to 128 characters and must not start with `aws:`\. The tag value can be up to 256 characters\.  *Required*: No  *Type*: List of Tag  *Update requires*: No interruption`Value`  Specifies a value of the API key\.  *Required*: No  *Type*: String  *Update requires*: Replacement## Return values### RefWhen you pass the logical ID of this resource to the intrinsic `Ref`function, `Ref`returns the API key ID, such as `m2m1k7sybf`\.For more information about using the `Ref`function, see Ref\.### Fn::GetAttThe `Fn::GetAtt`intrinsic function returns a value for a specified attribute of this type\. The following are the available attributes and sample return values\.For more information about using the `Fn::GetAtt`intrinsic function, see Fn::GetAtt\.#### `APIKeyId`  The ID for the API key\. For example: `abc123`\.## Examples### API KeyThe following example creates an API key and associates it with the `Test` stage of the `TestAPIDeployment` deployment\. To ensure that AWS CloudFormation creates the stage and deployment \(which are declared elsewhere in the same template\) before the API key, the example adds an explicit dependency on the deployment and stage\. Without this dependency, AWS CloudFormation might create the API key first, which would cause the association to fail because the deployment and stage wouldn't exist\.#### JSON```{    "ApiKey": {        "Type": "AWS::ApiGateway::ApiKey",        "DependsOn": [            "TestAPIDeployment",            "Test"        ],        "Properties": {            "Name": "TestApiKey",            "Description": "CloudFormation API Key V1",            "Enabled": true,            "StageKeys": [                {                    "RestApiId": {                        "Ref": "RestApi"                    },                    "StageName": "Test"                }            ]        }    }}```### Customer IDThe following example creates an API key, and enables you to specify a customer ID and whether to create a distinct ID\.#### JSON```{  "Parameters": {    "apiKeyName": {      "Type": "String"    },    "customerId": {      "Type": "String"    },    "generateDistinctId": {      "Type": "String"    }  },  "Resources": {    "ApiKey": {      "Type": "AWS::ApiGateway::ApiKey",      "Properties": {        "CustomerId": {          "Ref": "customerId"        },        "GenerateDistinctId": {          "Ref": "generateDistinctId"        },        "Name": {          "Ref": "apiKeyName"        }      }    }  }}```## See also+ apikey:create in the *Amazon API Gateway REST API Reference*[/INST]
"""

In [14]:
prompt = "<s>[INST] ec2 만들어줘 [/INST]"

In [16]:
with torch.no_grad():
    gened = model.generate(
            **tokenizer(prompt, return_tensors="pt", return_token_type_ids=False).to(
                device
            ),
            num_beams=1
        )

In [17]:
generated_text = tokenizer.decode(gened[0])

In [18]:
generated_text

'<s><s> [INST] ec2 만들어줘 [/INST] ec2 만들어줘 [INST] ec2 만들어줘 [INST] ec2 만들어줘 [INST] ec2 만들어줘 [INST] ec2 만들어줘 [INST] ec2 만들어줘 [INST] ec2 만들어줘 [INST] ec2 만들어줘 [INST] ec2 만들어줘 [INST] ec2 만들어줘 [INST] ec2 만들어줘 [INST] ec2 만들어줘 [INST] ec2 만들어줘 [INST] ec2 만들어줘 [INST] ec2 만들어줘 [INST] ec2 만들어줘 [INST] ec2 만들어줘 [INST] ec2 만들어줘 [INST] ec2 만들어줘 [INST] ec2 만들어줘 [INST] ec2 만들어줘 [INST] ec2 만들어줘 [INST] ec2 만들어줘 [INST] ec2 만들어줘 [INST] ec2 만들어줘 [INST] ec2 만들어줘 [INST] ec2 만들어줘 [INST] ec2 만들어줘 [INST] ec2 만들어줘 [INST] ec2 만들어줘 [INST] ec2 만들어줘 [INST] ec2 만들어줘 [INST] ec2 만들어줘 [INST] ec2 만들어줘 [INST] ec2 만들어줘 [INST] ec2 만들어줘 [INST] ec2 만들어줘 [INST] ec2 만들어줘 [INST] ec2 만들어줘 [INST] ec2 만들어줘 [INST] ec2 만들어줘 [INST] ec2 만들어줘 [INST] ec2 만들어줘 [INST] ec2 만들어줘 [INST] ec2 만들어줘 [INST] ec2 만들어줘 [INST] ec2 만들어줘 [INST] ec2 만들어줘 [INST] ec2 만들어줘 [INST] ec2 만들어줘 [INST] ec2 만들어줘 [INST] ec2 만들어줘 [INST] ec2 만들어줘 [INST] ec2 만들어줘 [INST] ec2 만들어줘 [INST] ec2 만들어줘 [INST] ec2 만들어줘 [INST] ec2 만들어줘 [INST] ec2 만들어줘 [INST] ec2 만들어줘 [INST] ec2 만들어줘

In [13]:
generated_text

'<s> \n<s> [INST]너는 지금부터 아래 내용에 따라 AWS CloudFormation JSON 템플릿을 만드는 역할을 할거야. 너한테 AWS CloudFormation의 템플릿 파일을 생성하기 위해 지시를 내리는 문장과 지시 문장과 관련된 AWS의 CloudFormation의 문서를 제공할거야. - AWS CloudFormation 템플릿의 예시: AWS CloudFormation 템플릿 형태에 대해 설명해줄게.    템플릿 구조    JSON    {    "AWSTemplateFormatVersion" : "version date",    "Description" : "JSON string",    "Resources" : {      set of resources    }  }  템플릿 섹션  AWSTemplateFormatVersion  AWSTemplateFormatVersion 섹션은 템플릿의 기능을 식별합니다. 최신 템플릿 포맷 버전은 2010-09-09이며 현재 유일한 유효 값입니다.  Description  템플릿의 Description 섹션(선택 사항)에 템플릿에 대한 설명을 지정합니다. 설명 선언 값은 0 ~ 1023바이트 길이의 리터럴 문자열이어야 합니다. 파라미터나 함수를 사용하여 설명을 지정할 수 없습니다. 다음 코드 조각은 설명 선언을 보여주는 예입니다.  Resources  필수 Resources 섹션은 Amazon EC2 인스턴스 또는 Amazon S3 버킷 등 스택에 포함시킬 AWS 리소스를 선언합니다.  Resources 섹션은 키 이름 Resources로 이루어집니다. 다음 가상 템플릿에는 Resources 섹션이 요약되어 있습니다.  JSON  "Resources" : {      "Logical ID" : {          "Type" : "Resource type",          "Properties" : {              Set of properties          }      }}    